In [113]:
# install beautifulsoup 4 and parser

! pip install beautifulsoup4
! pip install lxml


In [9]:
# import libraries

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [104]:
# get html from Wikipedia and parse the table

res =  requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content, 'lxml')
table = soup.find_all('table')[0]

#print(table)

In [113]:
#create a pandas dataframe from the parsed html table

df = pd.read_html(str(table))[0]
df.head(10)
df.shape


(287, 3)

In [114]:
#Clean the dataframe of Boroughs=Not assigned values, postcodes with multiple neigborhoods etc.

#drop rows with Borough = Not assigned
df.rename(columns={'District': 'Borough'}, inplace=True)
df.drop(df[df['Borough']=='Not assigned'].index, inplace=True)
#df.head(10)
df.shape



(210, 3)

In [108]:
#Combine postal code with multiple neighborhoods
#df.groupby(['Postcode','Borough'], as_index=False, sort=False).agg(lambda x:','.join(x))

df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list)
df = df.sample(frac=1).reset_index()
df['Neighbourhood'] = df['Neighbourhood'].str.join(',')
df.head()



,Postcode,Borough,Neighbourhood
0,M1R,Scarborough,"Maryvale,Wexford"
1,M5M,North York,"Bedford Park,Lawrence Manor East"
2,M4K,East Toronto,"The Danforth West,Riverdale"
3,M9W,Etobicoke,Northwest
4,M7R,Mississauga,Canada Post Gateway Processing Centre


In [109]:
#Assign value of Borough to Neighbourhood when Neighbourhood is 'Not assigned'

df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']


In [112]:
#Shape of dataframe after aggregating Neighbourhoods etc.
df.shape

(103, 3)

,Postcode,Borough,Neighbourhood
0,M1R,Scarborough,"Maryvale,Wexford"
1,M5M,North York,"Bedford Park,Lawrence Manor East"
2,M4K,East Toronto,"The Danforth West,Riverdale"
3,M9W,Etobicoke,Northwest
4,M7R,Mississauga,Canada Post Gateway Processing Centre
